Corriger une prédiction HTR
===
<br>

Nous reprenons ici la démarche exposée par Floriane Chiffoleau dans la documentation du projet [DAHN](https://github.com/FloChiff/DAHNProject/blob/8df8dfc6053a7dd57a6c5510d1e56bb336ce1d04/Project%20development/Documentation/Post-OCR%20correction%20for%20TEXT%20files.md).

# Analyser le contenu de la prédiction

Le script [spellcheckTexts.py](./py/spellcheckTexts.py) analyse chaque mot des prédictions HTR et génère pour chaque fichier XML à corriger un dictionnaire au format Json contenant des propositions de correction.

On s'assure dans un premier temps de disposer des dossiers nécessaires au stockage des fichiers :

In [2]:
from py.constantes import DICTPAGESNONCORR
print(f"On placera les dictionnaires de page au format Json dans le dossier : {DICTPAGESNONCORR}")

# La commande suivante crée le dossier s'il n'existe pas encore
!mkdir --parents {DICTPAGESNONCORR}

On placera les dictionnaires de page au format Json dans le dossier : ./py/dicos/aCorriger/


**On exécute le script [spellcheckTexts.py](https://github.com/sbiay/CdS-edition/blob/main/htr/py/spellcheckTexts.py)**. Le temps de calcul étant long (5 min environ), on peut commencer à corriger la première page terminée tandis que le script continue à tourner sur les autres :

In [3]:
!python3 py/spellcheckTexts.py

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0175.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-02_0175.json a été écrit correctement.

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0067.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-02_0067.json a été écrit correctement.

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0042.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-02_0042.json a été écrit correctement.

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0073.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-02_0073.json a été écrit correctement.

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0035.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-02_0035.json a été écrit correctement.

Le fichier ./predic-a-corriger/CdS02_Konv002-02_0041.xml est en cours de lecture.
=> Le diction

# Corriger les dictionnaires de pages

On corrige à la main les entrées du dictionnaire de chacun des fichiers générés dans le dossier [dicos/aCorriger](py/dicos/aCorriger).

Voici une série de cas particuliers accompagnés de leur méthode de résolution.

On entend par **forme** le résultat de la prédiction HTR et par **lemme** la correction adéquate ou les corrections possibles de la forme.

## Forme erronée et plusieurs propositions de correction

Lorsque les propositions de corrections sont multiples, on doit veiller à **placer en première position la correction souhaitée** :
```json
"fout": {
    "lem": [
        "faut",
        "font"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```

Il faut donc corriger pour obtenir :
```json
"fout": {
    "lem": [
        "font",
        "faut"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```

On peut également supprimer la correction non souhaitée :
```json
"fout": {
    "lem": [
        "font"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```


## Forme juste et proposition de correction fausse

Lorsque la forme prédite est juste mais que la proposition de correction n'est pas juste, il faut la neutraliser en remplaçant la proposition par **null**. Par exemple, le dictionnaire propose automatiquement :
```json
"Cet": {
    "lem": [
        "et"
        ],
    "ctxt": "CET ouvrage que je viens de publier"
}
```
Ici, "Cet" est juste, et la proposition "et" est fausse, on doit donc la remplacer ainsi :
```json
"Cet": {
    "lem": [
        null
        ],
    "ctxt": "CET ouvrage que je viens de publier"
}
```

## Forme juste et proposition de correction "null"

Il n'est pas nécessaire de transformer les mots justes dont le lemme est indiqué comme "null". Ils apparaîtront tels quels dans le fichier XML corrigé.

## Les noms propres

Les **patronymes** représentent une difficulté importante de la transcription. Pour la plupart, ils ont par bonheur été déjà identifiés dans les notices de l'inventaire. On se reportera donc aux notices du site web. On se reportera aux [outils du dossier donnees](https://github.com/sbiay/CdS-edition/tree/main/donnees#outils), et plus particulièrement au [notebook](https://github.com/sbiay/CdS-edition/blob/main/donnees/Obtenir_metadonnees_images.ipynb) dédié à la récupération des liens vers les notices de l'inventaire à partir d'un nom de fichier image.

## Sauvegarder le dictionnaire de page corrigé

Une fois les corrections faites, on copie le fichier dans un dossier dédié :

In [ ]:
from py.constantes import DICTPAGESCORR
print(f"On place les dictionnaires de pages corrigés dans le dossier : {DICTPAGESCORR}")

In [ ]:
# Si le dossier n'a pas encore été créé, on peut lancer la commande suivante pour le faire
!mkdir {DICTPAGESCORR}

# Appliquer le dictionnaire du projet aux fichiers XML

On procède à l'application des corrections grâce au script [text_correction_XML.py](./py/text_correction_XML.py) :

In [ ]:
!python3 py/textCorrection.py

Ce script ajoute automatiquement les corrections appliquées au fichier global [correctionsCDS.json](./py/dicos/correctionsCDS.json). Si d'autres corrections existent déjà pour les mêmes formes dans [correctionsCDS.json](./py/dicos/correctionsCDS.json), l'ensemble des corrections constitue une liste, réexploitée à la prochaine correction de page en proposant comme premier choix le mot le plus fréquent parmi les vérités de terrain.